In [ ]:
using _Data, _Math
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2, BenchmarkTools, Random

In [ ]:
@pyimport numpy
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.metrics as metrics

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/utils.jl")

In [ ]:
analyzer = "chuyu"

In [ ]:
data_info_dict = load("/home/byoungsoo/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

In [ ]:
keys(data_info_dict)

In [ ]:
job_names_ordered = [
 "corner_cue_rotation_whole"
 "landmark_removal"
 "boundary_morphing"
 "job_retangular_half_circle_rotation"

    
 "hedgehog_rotation_bottom_out"
 "hedgehog_landmark_removal"
 "hedgehog_circle"
 "corner_cue_circle"
    
]


job_names_figure = [
 "Chamber rotation"
 "Landmark removal"
 "Wall morphing"
    
 "Wall rotation"
    
 "No change"
 "Landmark removal"
 "Wall morphing"
 "Wall morphing + \n Landmark removal"
]


In [ ]:
fig_path = "/home/byoungsoo/Notebooks/project_place_cell/figures/output/review_figures/new_dropout"

In [ ]:
data_info_all = reduce(vcat, [data_info_dict[job_name] for job_name in keys(data_info_dict)]);

In [ ]:
# percentage of area overlap in PF
function PF_overlap(which_map_1, which_map_2)
    PF1 = map_field(which_map_1)
    PF2 = map_field(which_map_2)
    overlap_PF = PF1.*PF2
    union_PF = (PF1.+PF2 .> 0)
    
    size_unionPF = sum(union_PF)
    size_overlapPF = sum(overlap_PF)

    overlap_percentage = size_overlapPF/size_unionPF

    return overlap_percentage
end

function PF_corr(which_map_1, which_map_2)
    PF1 = map_field(which_map_1)
    PF2 = map_field(which_map_2)
    union_PF = (PF1.+PF2 .> 0)

    PF_map_1 = fill(NaN32, size(which_map_1))
    PF_map_1[union_PF] .= which_map_1[union_PF]
    PF_map_2 = fill(NaN32, size(which_map_2))
    PF_map_2[union_PF] .= which_map_2[union_PF]

    return corr_2d_original(PF_map_1,PF_map_2)
end

In [ ]:
data_info_dict["hedgehog_landmark_removal"]

In [ ]:
morphing_need_data = [data_info_dict["boundary_morphing"];data_info_dict["hedgehog_circle"];data_info_dict["corner_cue_circle"]]
morphing_need_filename = [data_info[3] for data_info in morphing_need_data]

In [ ]:
# PF overlap
fig = figure(figsize=(7,1.5))
tick_locs = []
tick_labels = []

trials_place_cell_1_tog = []
trials_place_cell_2_tog = []
trials_union_place_cell_tog = []
trials_original_PFcor_tog = []
trials_original_PFcov_tog = []
trials_suffled_PFcov_tog = []
trials_PFoverlap_tog = []

earlylate_place_cell_1_tog = []
earlylate_place_cell_2_tog = []
earlylate_union_place_cell_tog = []
earlylate_original_PFcor_tog = []
earlylate_original_PFcov_tog = []
earlylate_suffled_PFcov_tog = []
earlylate_PFoverlap_tog = []

for (i_job, job_name) in enumerate(job_names_ordered)
    data_info_all = data_info_dict[job_name]
    job_name = job_names_figure[i_job]
    
    println(job_name)
    println(i_job)
    
    trials_median_tog = []
    earlylate_median_tog = []
    
    flag = 0
    if job_name == "Chamber rotation"
        flag = 1
    end

    for which_data in 1:length(data_info_all)
        data_info = data_info_all[which_data]
    
        experiment_filename_1 = data_info[1]
        server_1 = data_info[2]
    
        experiment_filename_2 = data_info[3]
        server_2 = data_info[4]
        experimenter = data_info[end]

        if job_name == "Chamber rotation"
            continue
        end

        ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
        ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
        ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
        
        ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
        ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
        ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

        info_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
        file = h5open(info_filename, "r")
        whether_tel = HDF5.readmmap(file["whether_tel"])
        close(file) 

        all_files = readdir(data_path(ds_save_cy_1))
        which_file = [occursin(experiment_filename_1, all_files[i])*occursin("A_dF", all_files[i])*occursin("neuron", all_files[i]) for i in 1:length(all_files)]
        @assert(length(all_files[which_file]) == 1)
        save_file_name = all_files[which_file][1]
        info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
        file = h5open(info_filename, "r")
        place_map_all_1 = HDF5.readmmap(file["place_map_all"])
        specificity_1 = HDF5.readmmap(file["specificity"])
        specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
        specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
        stability_1 = HDF5.readmmap(file["stability_all"])
        close(file)    
    
        all_files = readdir(data_path(ds_save_cy_2))
        which_file = [occursin(experiment_filename_2, all_files[i])*occursin("A_dF", all_files[i])*occursin("neuron", all_files[i]) for i in 1:length(all_files)]
        @assert(length(all_files[which_file]) == 1)
        save_file_name = all_files[which_file][1]
        info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
        file = h5open(info_filename, "r")
        place_map_all_2 = HDF5.readmmap(file["place_map_all"])
        specificity_2 = HDF5.readmmap(file["specificity"])
        specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
        specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
        stability_2 = HDF5.readmmap(file["stability_all"])
        close(file)

        if flag == 1
            which_file = [occursin(experiment_filename_1, all_files[i])*occursin("A_dF", all_files[i])*occursin("neuron", all_files[i]) for i in 1:length(all_files)]          
            @assert(length(all_files[which_file]) == 1)
            save_file_name = all_files[which_file][1]
            info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
            file = h5open(info_filename, "r")
            place_map_all_2 = HDF5.readmmap(file["place_map_all"])
            close(file)
        end

        if experiment_filename_2 in morphing_need_filename
            println("true")
            all_files = readdir(data_path(ds_save_cy_2))
            which_file = [occursin("boundary_morph_matchangle_v2.h5", all_files[i]) for i in 1:length(all_files)]
            @assert(length(all_files[which_file]) == 1)
            save_file_name = all_files[which_file][1]
            info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
            file = h5open(info_filename, "r")
            mapped_map_all = HDF5.readmmap(file["mapped_map_all"])
            close(file)
        end
        
        place_cell_index_1 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01), findall(whether_tel));
        place_cell_index_2 = intersect(findall(specificity_population_z_2.>3), findall(specificity_shuffle_z_2.>5), findall(specificity_2.>0.01), findall(whether_tel));
        union_place_cell_index = union(place_cell_index_1,place_cell_index_2)
    
        append!(trials_place_cell_1_tog, [place_cell_index_1])
        append!(trials_place_cell_2_tog, [place_cell_index_2])
        append!(trials_union_place_cell_tog, [union_place_cell_index])

        shuffled_PFcov_all = Array{Any}(undef,length(specificity_population_z_1))
        original_PFcor_all = fill(NaN32,length(specificity_population_z_1))
        original_PFcov_all = fill(NaN32,length(specificity_population_z_1))
        PFoverlap_all = zeros(length(specificity_population_z_1))
        
        @showprogress for which_neuron in union_place_cell_index
            if experiment_filename_2 in morphing_need_filename
                s1_map = place_map_all_1[:,:,which_neuron]
                s2_map = mapped_map_all[:,:,which_neuron]
            else
                s1_map = place_map_all_1[:,:,which_neuron]
                s2_map = place_map_all_2[:,:,which_neuron]
            end
            original_PFcor = corr_2d_original(s1_map,s2_map)
            PFoverlap_all[which_neuron] = PF_overlap(s1_map,s2_map)
            original_PFcor_all[which_neuron] = original_PFcor
            
            # computing null distribution for the PF correlation
            valid_index = findall((.!isnan.(s1_map)).*(.!isnan.(s2_map)))
            if length(valid_index) == 0
                continue
            end
            valid_s1_map = s1_map[valid_index]
            valid_s2_map = s2_map[valid_index]
            original_PFcov = cov(valid_s1_map,valid_s2_map)
            
            # manual circular shuffle
            max_shift = length(valid_index)
            shift_steps = collect(1:max_shift)
    
            shuffled_PFcov = zeros(max_shift)
            for n in 1:max_shift
                shuffled_valid_s2_map = numpy.roll(valid_s2_map,n)
                shuffled_PFcov[n] = cov(valid_s1_map,shuffled_valid_s2_map)
            end
            original_PFcov_all[which_neuron] = original_PFcov
            shuffled_PFcov_all[which_neuron] = shuffled_PFcov
        end
        
        append!(trials_original_PFcor_tog, [original_PFcor_all])
        append!(trials_original_PFcov_tog, [original_PFcov_all])
        append!(trials_suffled_PFcov_tog, [shuffled_PFcov_all])
        append!(trials_PFoverlap_tog, [PFoverlap_all])

        file_folder_1 = joinpath(data_path(ds_save_cy_1), "place_cell_windows")
        all_files = readdir(file_folder_1)
        long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
        spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
        candidate_filename = long_name_files[spatial_info_index]
        which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
        save_file_name = candidate_filename[which_file][1]
        info_filename = joinpath(data_path(ds_save_cy_1),"place_cell_windows", save_file_name)
        file = h5open(info_filename, "r")
        place_map_all_1 = HDF5.readmmap(file["place_map_all"])
        specificity_1 = HDF5.readmmap(file["specificity"])
        specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
        specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
        stability_1_early = HDF5.readmmap(file["stability_all"])
        close(file)
            
        save_file_name = candidate_filename[which_file][2]
        info_filename = joinpath(data_path(ds_save_cy_1),"place_cell_windows", save_file_name)
        file = h5open(info_filename, "r")
        place_map_all_2 = HDF5.readmmap(file["place_map_all"])
        specificity_2 = HDF5.readmmap(file["specificity"])
        specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
        specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
        stability_1_late = HDF5.readmmap(file["stability_all"])
        close(file)

        place_cell_index_1 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01), findall(whether_tel));
        place_cell_index_2 = intersect(findall(specificity_population_z_2.>3), findall(specificity_shuffle_z_2.>5), findall(specificity_2.>0.01), findall(whether_tel));
        union_place_cell_index = union(place_cell_index_1,place_cell_index_2)
        
        append!(earlylate_place_cell_1_tog, [place_cell_index_1])
        append!(earlylate_place_cell_2_tog, [place_cell_index_2])
        append!(earlylate_union_place_cell_tog, [union_place_cell_index])

        shuffled_PFcov_all = Array{Any}(undef,length(specificity_population_z_1))
        original_PFcor_all = fill(NaN32,length(specificity_population_z_1))
        original_PFcov_all = fill(NaN32,length(specificity_population_z_1))
        PFoverlap_all = zeros(length(specificity_population_z_1))
        
        @showprogress for which_neuron in union_place_cell_index
            s1_map = place_map_all_1[:,:,which_neuron]
            s2_map = place_map_all_2[:,:,which_neuron]
            
            original_PFcor = corr_2d_original(s1_map,s2_map)
            PFoverlap_all[which_neuron] = PF_overlap(s1_map,s2_map)
            original_PFcor_all[which_neuron] = original_PFcor
            
            # computing null distribution for the PF correlation
            valid_index = findall((.!isnan.(s1_map)).*(.!isnan.(s2_map)))
            if length(valid_index) == 0
                continue
            end
            valid_s1_map = s1_map[valid_index]
            valid_s2_map = s2_map[valid_index]
            original_PFcov = cov(valid_s1_map,valid_s2_map)
            
            # manual circular shuffle
            max_shift = length(valid_index)
            shift_steps = collect(1:max_shift)
    
            shuffled_PFcov = zeros(max_shift)
            for n in 1:max_shift
                shuffled_valid_s2_map = numpy.roll(valid_s2_map,n)
                shuffled_PFcov[n] = cov(valid_s1_map,shuffled_valid_s2_map)
            end
            original_PFcov_all[which_neuron] = original_PFcov
            shuffled_PFcov_all[which_neuron] = shuffled_PFcov
        end
        
        append!(earlylate_original_PFcor_tog, [original_PFcor_all])
        append!(earlylate_original_PFcov_tog, [original_PFcov_all])
        append!(earlylate_suffled_PFcov_tog, [shuffled_PFcov_all])
        append!(earlylate_PFoverlap_tog, [PFoverlap_all])
    end
end

In [ ]:
# trials_two_sided_p_values_tog = []
trials_one_sided_p_values_tog = []
trials_z_scores_tog = []

earlylate_two_sided_p_values_tog = []
earlylate_one_sided_p_values_tog = []
earlylate_z_scores_tog = []

@showprogress for exp_idx in 1:length(trials_original_PFcor_tog)
    trials_original_PFcor_all = trials_original_PFcor_tog[exp_idx]
    trials_original_PFcov_all = trials_original_PFcov_tog[exp_idx]
    trials_shuffled_PFcov_all = trials_suffled_PFcov_tog[exp_idx]
    
    trials_two_sided_p_values = fill(NaN32, length(trials_place_cell_1_tog[exp_idx]))
    trials_one_sided_p_values = fill(NaN32, length(trials_place_cell_1_tog[exp_idx]))
    trials_z_scores = fill(NaN32, length(trials_place_cell_1_tog[exp_idx]))
    for (i,which_neuron) in enumerate(trials_place_cell_1_tog[exp_idx])
        test_PFcov = trials_original_PFcov_all[which_neuron]
        null_PFcov_dist = []
        try null_PFcov_dist = trials_shuffled_PFcov_all[which_neuron]
        catch
            continue
        end
    
        null_mean = nanmean(null_PFcov_dist)
        null_std = nanstd(null_PFcov_dist)
        if isfinite(test_PFcov)
            # P(null_PFcov >= test_PFcov)
            prob_1 = length(findall(null_PFcov_dist .>= test_PFcov)) / length(null_PFcov_dist)
            # P(null_PFcov <= test_PFcov)
            prob_2 = length(findall(null_PFcov_dist .<= test_PFcov)) / length(null_PFcov_dist)
            # p_values
            # two-tailed p-value
            trials_two_sided_p_values[i] = 2*min(prob_1,prob_2)
            # one-tailed p-value
            trials_one_sided_p_values[i] = prob_1
            # z-score
            trials_z_scores[i] = (test_PFcov - null_mean) / null_std
        end
    end

    append!(trials_two_sided_p_values_tog, [trials_two_sided_p_values])
    append!(trials_one_sided_p_values_tog, [trials_one_sided_p_values])
    append!(trials_z_scores_tog, [trials_z_scores])
end

@showprogress for exp_idx in 1:length(earlylate_original_PFcor_tog)
    earlylate_original_PFcor_all = earlylate_original_PFcor_tog[exp_idx]
    earlylate_original_PFcov_all = earlylate_original_PFcov_tog[exp_idx]
    earlylate_shuffled_PFcov_all = earlylate_suffled_PFcov_tog[exp_idx]
    
    earlylate_two_sided_p_values = fill(NaN32, length(earlylate_place_cell_1_tog[exp_idx]))
    earlylate_one_sided_p_values = fill(NaN32, length(earlylate_place_cell_1_tog[exp_idx]))
    earlylate_z_scores = fill(NaN32, length(earlylate_place_cell_1_tog[exp_idx]))
    for (i,which_neuron) in enumerate(earlylate_place_cell_1_tog[exp_idx])
        test_PFcov = earlylate_original_PFcov_all[which_neuron]
        null_PFcov_dist = []
        try null_PFcov_dist = earlylate_shuffled_PFcov_all[which_neuron]
        catch
            continue
        end
        null_mean = nanmean(null_PFcov_dist)
        null_std = nanstd(null_PFcov_dist)
        if isfinite(test_PFcov)
            # P(null_PFcov >= test_PFcov)
            prob_1 = length(findall(null_PFcov_dist .>= test_PFcov)) / length(null_PFcov_dist)
            # P(null_PFcov <= test_PFcov)
            prob_2 = length(findall(null_PFcov_dist .<= test_PFcov)) / length(null_PFcov_dist)
            # p_values
            # two-tailed p-value
            earlylate_two_sided_p_values[i] = 2*min(prob_1,prob_2)
            # one-tailed p-value
            earlylate_one_sided_p_values[i] = prob_1
            # z-score
            earlylate_z_scores[i] = (test_PFcov - null_mean) / null_std
        end
    end

    append!(earlylate_two_sided_p_values_tog, [earlylate_two_sided_p_values])
    append!(earlylate_one_sided_p_values_tog, [earlylate_one_sided_p_values])
    append!(earlylate_z_scores_tog, [earlylate_z_scores])
end

In [ ]:
jldopen(joinpath(filepath, "place_cell_dropout.jld"), "w") do file
    write(file, "earlylate_PFoverlap_tog", earlylate_PFoverlap_tog)
    write(file, "trials_PFoverlap_tog", trials_PFoverlap_tog)

    write(file, "earlylate_one_sided_p_values_tog", earlylate_one_sided_p_values_tog)
    write(file, "trials_one_sided_p_values_tog", trials_one_sided_p_values_tog)

    write(file, "earlylate_original_PFcor_tog", earlylate_original_PFcor_tog)
    write(file, "trials_original_PFcor_tog", trials_original_PFcor_tog)

    write(file, "earlylate_place_cell_1_tog", earlylate_place_cell_1_tog)
    write(file, "earlylate_place_cell_2_tog", earlylate_place_cell_2_tog)
    write(file, "earlylate_union_place_cell_tog", earlylate_union_place_cell_tog)
    
    write(file, "trials_place_cell_1_tog", trials_place_cell_1_tog)
    write(file, "trials_place_cell_2_tog", trials_place_cell_2_tog)
    write(file, "trials_union_place_cell_tog", trials_union_place_cell_tog)

    write(file, "job_names_ordered", job_names_ordered)
    write(file, "data_info_dict", data_info_dict)
    write(file, "job_names_figure", job_names_figure)
    
end;